In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from datetime import datetime

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Course 1 : Working with Time Series in Pandas**

# **How to use dates & times with pandas**

In [ ]:
time_stamp = pd.Timestamp(2021,4,17)
pd.Timestamp("2021-4-17") == time_stamp

In [ ]:
time_stamp

In [ ]:
time_stamp.year

In [ ]:
time_stamp.day

In [ ]:
time_stamp.day_name()

In [ ]:
time_stamp.day_of_week

In [ ]:
time_stamp.day_of_year

In [ ]:
time_stamp.days_in_month

In [ ]:
time_stamp.is_year_end

In [ ]:
time_stamp.quarter

In [ ]:
time_stamp.value

In [ ]:
period = pd.Period("2021-04")
period

In [ ]:
period.asfreq("D")

In [ ]:
period.to_timestamp().to_period('M')

In [ ]:
period+9

In [ ]:
index = pd.date_range(start="2021-04-17", periods=12, freq= "M")

In [ ]:
index

In [ ]:
index[0]

In [ ]:
index[10]

In [ ]:
index.to_period()

In [ ]:
index2 = pd.date_range(start= "2021-04-17", periods=12, freq="Q")
index2

In [ ]:
index3 = pd.date_range(start= "2021-04-17", periods=12, freq="A")
index3

In [ ]:
index4 = pd.date_range(start= "2021-04-17", periods=14, freq="H")
index4

In [ ]:
index5 = pd.date_range(start= "2021-04-17", periods=12, freq="W")
index5

In [ ]:
index6 = pd.date_range(start= "2021-04-17", periods=62, freq="S")
index6

In [ ]:
pd.DataFrame({"Data":index}).info()

In [ ]:
data = np.random.random(size=(12,2))
pd.DataFrame(data=data,index=index).info()

In [ ]:
data

# **Indexing & resampling time series**

In [ ]:
google = pd.read_csv("../input/googlestockpricing/Google.csv")
google.head()

In [ ]:
google.info()

In [ ]:
#Converting string dates to datetime64
google.Date = pd.to_datetime(google.Date)
google.info()

In [ ]:
google.set_index("Date", inplace=True)
google.info()

In [ ]:
google = google.rename(columns={"High": "price"})

In [ ]:
google.head()

In [ ]:
#Plotting the Google stock time series
google.price.plot(title="Google Stock Price")
plt.tight_layout(); plt.show()

In [ ]:
google.plot(subplots=True)
plt.show()

In [ ]:
google.head()

In [ ]:
#Partial string indexing
google.loc["2015"].info()

In [ ]:
google['2015-3': '2016-2'].info()

In [ ]:
google.loc["2015-4-23","price"]

In [ ]:
#.asfreq(): set frequency
google.asfreq('D').info()

In [ ]:
google.asfreq('D').head()

In [ ]:
google.asfreq('M').head()

In [ ]:
google.asfreq('A')

In [ ]:
#.asfreq(): reset frequency
google = google.asfreq('B') # Change to calendar day frequency
google.info()

In [ ]:
google[google.price.isnull()]

In [ ]:
"""# Create dataframe prices here
prices = pd.DataFrame()

# Select data for each year and concatenate with prices here 
for year in ['2013', '2014', '2015']:
    price_per_year = google.loc[year, ['price']].reset_index(drop=True)
    price_per_year.rename(columns={'price': year}, inplace=True)
    prices = pd.concat([prices, price_per_year], axis=1)

# Plot prices
prices.plot()
plt.show()"""

# **Lags, changes, and returns for stock price series**

In [ ]:
google = pd.read_csv('../input/googlestockpricing/Google.csv', parse_dates=['Date'], index_col='Date')
google.info()

In [ ]:
google.drop(google.columns[2:12], axis=1, inplace=True)

In [ ]:
google.drop(google.columns[0], axis=1, inplace=True)

In [ ]:
google = google.rename(columns={"High": "price"})

In [ ]:
google.head()

In [ ]:
#.shift(): Moving data between past & future
google["shifted"] = google.price.shift() #defaults to periods=1
google.head()

In [ ]:
google["lagged"] = google.price.shift(periods=-1)
google.tail()

In [ ]:
#Calculate one-period percent change
#x_t/x_(t-1)
google["change"] = google.price.div(google.shifted)
google.head()

In [ ]:
google['return'] = google.change.sub(1).mul(100)
google[['price', 'shifted', 'change', 'return']].head(3)

In [ ]:
#.diff(): built-in time-series change
#x_t - x_(t-1)
google['diff'] = google.price.diff()
google[['price', 'diff']].head(3)

In [ ]:
#pct_change():built-intime-series%change
google["pct_change"] = google.price.pct_change().mul(100)
google[['price', 'return', 'pct_change']].head(3)

* Percent change for two adjacent periods
* $x_t/x_(t-1)$

In [ ]:
#Lookingahead:Getmulti-periodreturns
google["return_3d"] = google.price.pct_change(periods=3).mul(100)
google[['price', "pct_change", 'return_3d']].head()

# **Course 2 : Basic Time Series Metrics & Resampling**

# **Compare time series growth rates**

In [ ]:
#Normalizing a single series
google = pd.read_csv('../input/googlestockpricing/Google.csv', parse_dates=['Date'], index_col='Date')
google.drop(google.columns[2:12], axis=1, inplace=True)
google.drop(google.columns[0], axis=1, inplace=True)
google = google.rename(columns={"High": "price"})
google.head(3)

In [ ]:
first_price = google.price.iloc[0]
first_price

In [ ]:
first_price == google.loc["2004-08-19","price"]

In [ ]:
normalized = google.price.div(first_price).mul(100)
normalized.plot(title="Google Normalized Series")

> 1000 percantage points ( 1200 - 200)

In [ ]:
#Normalizing multiple series
prices = pd.read_csv("../input/gafa-stock-prices/YahooFinance - GAFA stock prices.csv", parse_dates=["date"],index_col="date")
prices.drop(prices.columns[5:8], axis=1, inplace=True)
prices.drop(prices.columns[0:2], axis=1, inplace=True)
prices = prices.rename(columns={"high": "AAPL","low": "GOOG","close":"YHOO"})
prices.head(3)

In [ ]:
prices.iloc[0]

In [ ]:
normalized2 = prices.div(prices.iloc[0])
normalized2.head(3)

In [ ]:
#Comparing with a benchmark
index = pd.read_csv("../input/gafa-stock-prices/YahooFinance - GAFA stock prices.csv", parse_dates=["date"],index_col="date")
index.drop(index.columns[5:8], axis=1, inplace=True)
index.drop(index.columns[0:1], axis=1, inplace=True)
index = index.rename(columns={"open":"AAPL","high": "GOOG","low": "YHOO","close":"SP500",})
index.head()

In [ ]:
index.info()

In [ ]:
prices = pd.concat([prices,index],axis=1).dropna()
prices.info()

In [ ]:
prices.head(1)

In [ ]:
normalized3 = prices.div(prices.iloc[0]).mul(100)
normalized3.info()

In [ ]:
normalized3.head()

In [ ]:
normalized3.plot()

# **Changing the time series frequency: resampling**

* Upsampling: fill or interpolate missing data
* Downsampling: aggregate existing data

In [ ]:
#Getting started: quarterly data
dates = pd.date_range(start="2020",periods=4,freq="Q")
data=range(1,5)
quarterly=pd.Series(data=data,index=dates)
quarterly

In [ ]:
#Upsampling: quarter => month
monthly = quarterly.asfreq("M") # to month-end frequency
monthly

> Upsampling creates missing values

In [ ]:
monthly = monthly.to_frame("baseline") # to DataFrame

In [ ]:
#Upsampling: fill methods
monthly["ffill"] = quarterly.asfreq("M",method="ffill")
monthly["bfill"] = quarterly.asfreq("M",method="bfill")
monthly["value"] = quarterly.asfreq("M",fill_value=0)

* bfill : back fill
* ffill : forward fill

In [ ]:
monthly.head(10)

In [ ]:
#Add missing months: .reindex()
dates = pd.date_range(start="2020",periods=13,freq="M")
quarterly=quarterly.reindex(dates)
quarterly

# **Upsampling & interpolation with .resample()**

In [ ]:
#Getting started: monthly unemployment rate
unrate = pd.read_csv("../input/oecd-unemployment-rate-2020/2020 unemployment.csv",parse_dates=["TIME"])
unrate = unrate.rename(columns={"TIME": "Date"})
unrate.set_index("Date",inplace=True)
unrate.drop(unrate.columns[0:5], axis=1, inplace=True)
unrate.drop(unrate.columns[-1], axis=1, inplace=True)
unrate.head()

In [ ]:
unrate.info()

In [ ]:
#Resampling Period & Frequency Offsets


![frequency offset](https://www.kaggle.com/mehmetmarcelo/frequency-offsets?select=Screenshot_1.png)